## Runing psi-blast
This script runs psi-blast search by random protein of given cog in cogdb

In [6]:
# importing all modules and settings
import os
import pandas as pd
import numpy as np
from Bio.Blast.Applications import NcbipsiblastCommandline
from Bio.Align.Applications import MuscleCommandline
from Bio import Entrez
from Bio import SearchIO
from Bio import SeqIO
from Bio.Blast import NCBIXML
from collections import defaultdict
import pprint


Entrez.email = "isnicsi@gmail.com"
Entrez.api_key = "86bf08a1f3f3f3135f053107c3fa228c9508"

# These are the COGs suggested by correlation analysis
cogs = [
    "COG1563",
    "COG1269",
    "COG1827",
    "COG4945",
    "COG0636",
    "COG2426",
    "COG1906",
    "COG4769",
    "COG1822",
    "COG4720",
    "COG1852",
    "COG2456",
    "COG1883",
    "COG4708",
    "COG1750",
    "COG4035",
    "COG4042",
    "COG1784",
    "COG2034",
    "COG4039",
    "COG4038",
    "COG4907",
    "COG4040",
    "COG4037",
    "COG4036",
    "COG4041",
    "COG4078",
    "COG1933",
    "COG0650",
    "COG4059",
    "COG4060",
    "COG2245",
    "COG4061",
    "COG0170",
    "COG1456",
    "COG1800",
    "COG3872",
    "COG1284",
    "COG4743",
    "COG1916",
    "COG5306",
    "COG2035",
    "COG1967",
    "COG1340",
    "COG3641",
    "COG1422",
    "COG4026",
    "COG4658",
    "COG2461",
    "COG3889",
    "COG4660",
    "COG4657",
    "COG4025",
    "COG1811",
    "COG1963",
    "COG3086",
    "COG1361",
    "COG3356",
    "COG2155",
    "COG3601",
    "COG1824",
    "COG1480",
    "COG0619",
    "COG3371",
    "COG1955",
    "COG2431",
    "COG5625",
    "COG3314",
    "COG2512",
    "COG2510",
    "COG4640",
    "COG1863",
    "COG3815",
    "COG4046",
    "COG1930",
    "COG4603",
    "COG1470",
    "COG1079",
    "COG2383",
    "COG3330",
    "COG4827",
    "COG1624",
    "COG0530",
    "COG1006",
    "COG0310",
    "COG4346",
    "COG4089",
    "COG2059",
    "COG4906",
    "COG1347",
    "COG1814"
]  # specify cogs
found_cogs = defaultdict(set) # here all found cogs will be stored
psi_iter = 10
evalt = 1e-3
max_seqs = 500

In [2]:
# defining necessary functions
def fetch_seq(yp_id, label=""):    
    with Entrez.efetch(db="protein", rettype="fasta", retmode="text", id=yp_id) as handle:
        seq = SeqIO.read(handle, "fasta")
        seq.id += "_" + label
        return seq


def rename_seq(cogdb, seq):
    try:
        newid = cogdb.loc[seq.id.split("|")[-1]][6]
    except KeyError:
        newid = "0"
    if isinstance(newid, pd.Series):
        newid = "_".join(newid)
    found_cogs[cog].add(newid)
    seq.id = newid
    return seq

    
def rename_rec(cogdb, rec):
    return list(map(lambda x: rename_seq(cogdb, x[0].hit), rec))

In [3]:
# loading COG database
cogdb = pd.read_csv('/home/ilbumi/united_data_lib/cog/cog2003-2014.csv', header=None, index_col=0)

In [ ]:
# perform psi-blast
for cog in cogs:
    print("Processing "+cog)
    sample = list(map(lambda x: fetch_seq(x, label=cog), cogdb[cogdb[6]==cog].sample(1).index))
    os.makedirs(cog, exist_ok=True)
    SeqIO.write(sample, "{}/{}_query.fasta".format(cog, cog), "fasta")
    print("\tStarting PSI-BLAST...")
    cline = NcbipsiblastCommandline(
        query="{}/{}_query.fasta".format(cog, cog),
        db="711proteomes",
        outfmt=5,
        out="{}/{}_results.xml".format(cog, cog),
        evalue=evalt,
        max_target_seqs=1000,
        num_iterations=psi_iter,
        out_pssm="{}/{}_pssm.txt".format(cog, cog),
        num_threads=3
    )
    cline()
    print("\tPSI-BLAST done!")
    
    records = list(SearchIO.parse("{}/{}_results.xml".format(cog, cog), "blast-xml"))
    results = rename_rec(cogdb, records[-1])
    SeqIO.write(results, "results/{}_results.fasta".format(cog, cog), "fasta")

In [5]:
# printing found COGs
names = pd.read_table("/home/ilbumi/united_data_lib/cog/cognames2003-2014.tab", index_col=0)


def name_cogs(s):
    res = []
    for cog in s:
        if cog == '0':
            sr = ("UnCOGged")
        elif "_" in cog:
            sr = tuple(map(lambda x: names.loc[x]["name"], cog.split("_")))
        else:
            sr = (names.loc[cog]["name"])
        res.append(sr)
    return list(zip(s, res))
 
    
for k, v in found_cogs.items():
    print(k)
    found_names = name_cogs(v)
    for name in found_names:
        print("\t{}\t{}".format(*name))

/home/ilbumi/anaconda3/envs/sci/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


COG1563
	COG2111	Multisubunit Na+/H+ antiporter, MnhB subunit
	COG1563	Uncharacterized MnhB-related membrane protein
	COG1009_COG2111_COG2111	('NADH:ubiquinone oxidoreductase subunit 5 (chain L)/Multisubunit Na+/H+ antiporter, MnhA subunit', 'Multisubunit Na+/H+ antiporter, MnhB subunit', 'Multisubunit Na+/H+ antiporter, MnhB subunit')
	COG0839	NADH:ubiquinone oxidoreductase subunit 6 (chain J)
	COG1563_COG4623	('Uncharacterized MnhB-related membrane protein', 'Membrane-bound lytic murein transglycosylase MltF')
	COG0834	ABC-type amino acid transport/signal transduction system, periplasmic component/domain
	COG0839_COG0839	('NADH:ubiquinone oxidoreductase subunit 6 (chain J)', 'NADH:ubiquinone oxidoreductase subunit 6 (chain J)')
	COG1009_COG2111	('NADH:ubiquinone oxidoreductase subunit 5 (chain L)/Multisubunit Na+/H+ antiporter, MnhA subunit', 'Multisubunit Na+/H+ antiporter, MnhB subunit')
COG1269
	COG0004_COG3706	('Ammonia channel protein AmtB', 'Two-component response regulator, Pl

In [5]:
cog = "COG4720"
print("Processing "+cog)
sample = list(map(lambda x: fetch_seq(x, label=cog), cogdb[cogdb[6]==cog].sample(1).index))
os.makedirs(cog, exist_ok=True)
SeqIO.write(sample, "{}/{}_query.fasta".format(cog, cog), "fasta")
print("\tStarting PSI-BLAST...")
cline = NcbipsiblastCommandline(
    query="{}/{}_query.fasta".format(cog, cog),
    db="711proteomes",
    outfmt=5,
    out="{}/{}_results.xml".format(cog, cog),
    evalue=evalt,
    max_target_seqs=5000,
    num_iterations=psi_iter,
    out_pssm="{}/{}_pssm.txt".format(cog, cog),
    num_threads=3
)
cline()
print("\tPSI-BLAST done!")

records = list(SearchIO.parse("{}/{}_results.xml".format(cog, cog), "blast-xml"))
results = rename_rec(cogdb, records[-1])
SeqIO.write(results, "results/{}_results.fasta".format(cog, cog), "fasta")

Processing COG4720
	Starting PSI-BLAST...
	PSI-BLAST done!


639

In [6]:
names = pd.read_table("/home/ilbumi/united_data_lib/cog/cognames2003-2014.tab", index_col=0)


def name_cogs(s):
    res = []
    for cog in s:
        if cog == '0':
            sr = ("UnCOGged")
        elif "_" in cog:
            sr = tuple(map(lambda x: names.loc[x]["name"], cog.split("_")))
        else:
            sr = (names.loc[cog]["name"])
        res.append(sr)
    return list(zip(s, res))
 
    
for k, v in found_cogs.items():
    print(k)
    found_names = name_cogs(v)
    for name in found_names:
        print("\t{}\t{}".format(*name))

/home/ilbumi/anaconda3/envs/sci/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


COG4720
	0	UnCOGged
	COG2202_COG3275_COG5002	('PAS domain', 'Sensor histidine kinase, LytS/YehU family', 'Signal transduction histidine kinase')
	COG4191_COG4251	('Signal transduction histidine kinase regulating C4-dicarboxylate transport system', 'Bacteriophytochrome (light-regulated signal transduction histidine kinase)')
	COG2199	GGDEF domain, diguanylate cyclase (c-di-GMP synthetase) or its enzymatically inactive variants
	COG2199_COG2202_COG2206_COG3275	('GGDEF domain, diguanylate cyclase (c-di-GMP synthetase) or its enzymatically inactive variants', 'PAS domain', 'HD-GYP domain, c-di-GMP phosphodiesterase class II (or its inactivated variant)', 'Sensor histidine kinase, LytS/YehU family')
	COG3275	Sensor histidine kinase, LytS/YehU family
	COG0784_COG4191_COG4191	('CheY chemotaxis protein or a CheY-like REC (receiver) domain', 'Signal transduction histidine kinase regulating C4-dicarboxylate transport system', 'Signal transduction histidine kinase regulating C4-dicarboxylate tran